# Week 8: Motion Detection

<font size="6"> Laboratory 5 </font> <br>
<font size="3"> Last updated July 25, 2023 </font>

## <span style="color:orange;"> 00. Content </span>

<font size="5"> Mathematics </font>
- N/A
     
<font size="5"> Programming Skills </font>
- Functions and Modules
    
<font size="5"> Embedded Systems </font>
- Thonny and Micropython

## <span style="color:orange;"> 0. Required Hardware </span>

- Raspberry Pi Pico
- Breadboard
- USB connector
- Camera (Arducam HM01B0) 
- 8 Wires



<h3 style="background-color:lightblue"> Write your name and email below: </h3>

**Name:** Shiva Sai Vummaji

**Email:** svummaji@purdue.edu

*You should work in groups of 2-3 for this lab.*
<h3 style="background-color:lightblue"> Write your groupmates' names below: </h3>

**Groupmate(s):** Dhruv and Shawanwit



In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2

## <span style="color:orange;"> 1. Background Subtraction </span>

There is a popular image processing technique called background subtraction. The most effective way to utilize background subtraction for motion detection is when:

1. the camera is stationary
2. the background is static
3. the lighting conditions do not change
4. there is minimal noise in the image

One scenario where all of these conditions are met is in production/manufacturing lines. In controlled indoor environments, long-term changes are rare.

Essentially, background subtraction allows for the separation of the background from the foreground in an image. For instance, if a camera is set up to monitor a doorway, most of the time, nothing is in motion within the frame. By selecting an initial reference frame, we can compare all subsequent frames to the reference frame.

In [2]:
vid = cv2.VideoCapture('test_vid.mov') 
height = vid.get(cv2.CAP_PROP_FRAME_HEIGHT)
width  = vid.get(cv2.CAP_PROP_FRAME_WIDTH) 
scale = 0.125                                           # smaller scale for faster computations
new_size = (int(width*scale),int(height*scale))
frames = []                                             # saving frames to a list so that you can try
                                                        # methods quickly without reloading the video
while vid.isOpened():   
    success, frame = vid.read()
    if not success:
        print("Unable to read frame. Exiting ...")
        break
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    frame = cv2.resize(frame,dsize=new_size)
    frames.append(frame)
    cv2.imshow('frame', frame)                          # display grayscaled video resized, no other alterations
    if cv2.waitKey(25) == ord('q'):                     # press Q on keyboard to stop
        break
vid.release()
cv2.destroyAllWindows()

2025-03-05 14:53:59.410 python[98462:7028281] +[IMKClient subclass]: chose IMKClient_Modern
2025-03-05 14:53:59.410 python[98462:7028281] +[IMKInputSession subclass]: chose IMKInputSession_Modern


Background subtraction refers to a whole class of methods. However, for the scope of this lab, we will only be using one specific method. To demonstrate an example, we will utilize the same test video from the previous lab. To keep things concise, we won't delve into the detailed workings of `cv2.createBackgroundSubtractorMOG2()`. Generally, this method is slightly more robust against illumination changes. The output of the background subtraction function is known as a *mask*, which indicates that **the output is a binary image**. Pixel values in the mask are either 0 (representing the background) or 255 (representing the foreground).

Run the cell to observe the results!

In [4]:
fgbg = cv2.createBackgroundSubtractorMOG2(detectShadows=False)      # define this before looping through the frames

for f in frames:
    img = fgbg.apply(f)
    cv2.imshow('background subtracted frame',img)
    if cv2.waitKey(25) == ord('q'):                                 # press Q on keyboard to stop
        break
cv2.destroyAllWindows()

## <span style="color:orange;"> 2. Motion Detection and Localization </span>


### <span style="color:red"> Exercise 1</span>

Suppose we apply background subtraction to each video frame and divide the frame into an $M\times N$ grid, resulting in a total of $MN$ blocks. Can you consider a (mathematical) rule to determine whether there is movement in a specific block?

*Hint: Would the proportion of certain grayscale values be high or low when there is motion?*

<h3 style="background-color:lightblue"> Write Answers for Exercise 1 Below </h3>

Yes, we can consider a mathematical rule to determine the movement in a specific block. I can utilize the magnitudes of optical flow vectors (formula is by taking derivative of sqrt(u^2 + v^2) where u and v represent the optical flow vector using openCV). I can also utilize Fréchet Video Motion Distance formula, which ||μ_real - μ_gen||^2 + Tr(Σ_real + Σ_gen - 2(Σ_real * Σ_gen)^(1/2)). In this formula, μ_real and μ_gen refer to mean vectors of the motion features extracted from the real and generated videos. Σ_real and Σ_gen are covariance matrices.

## <span style="color:orange;"> 3. Connecting the Camera </span>

This time, we will record our own videos using the Arducam HM01B0, which is a small camera that can be connected to the Pico. 

### Wiring Instructions

Please ensure that your microcontroller is not connected to the computer while you are wiring components together. If you are unsure about your wiring, please consult the instructor. Use your jumper wires to establish the following connections:

| HM01B0 | Pico |
|--------|------|
| VCC    | 3V3  |
| SCL    | GP5  |
| SDA    | GP4  |
| VSYNC  | GP16 |
| HREF   | GP15 |
| PCLK   | GP14 |
| DO     | GP6  |
| GND    | GND  |

Here is an image of the completed breadboard:

![img](https://raw.githubusercontent.com/TheDataScienceLabs/DSLab_Probability/main/book/labs/shared_files/connecting_the_camera/HM01B0_and_pico.png)

<!-- ![img](camera.jpg)

*Wiring the Arducam HM01B0 camera* 

![img](camerawiring.jpg)

*Connections to the PICO on breadboard* -->

To find the names of the pins on the Raspberry Pi Pico, you can refer to its pinout diagram located [here](https://datasheets.raspberrypi.com/pico/Pico-R3-A4-Pinout.pdf) or in the Extra Materials section. The HM01B0, on the other hand, should have its pins labeled.

After confirming that the wiring is correct, press and hold the BOOTSEL button on the Pico while plugging it in. Download the [arducam.uf2](https://github.com/TheDataScienceLabs/DSLab_Probability/blob/main/book/labs/shared_files/connecting_the_camera/arducam.uf2) file and copy it onto the Pico's drive using your computer's file manager (it should be listed as an external drive: "RPI-RP2") and not with Thonny. Once the file transfer is complete, the Pico will automatically disconnect, and its LED will start blinking rapidly. 

Once the Pico has been successfully connected, please execute the following cell to ensure that we have successfully detected the Pico.

### <span style="color:red"> Exercise 2</span>

Using your camera, locate the area of motion in the video frame in real time (or near real time). You can employ any method to accomplish this, whether it involves background subtraction, your solution to the previous exercise, a combination of the two, or any other approach you can think of. Determine how to visually indicate the detected movement on the camera, such as printing a statement in Python or drawing a rectangle on the frame.

Display all the code you used in a cell below.

*Note: If you encounter significant hardware issues, you can try applying your method to the test video instead.*

<h3 style="background-color:lightblue"> Write Answers for Exercise 2 Below </h3>

In [7]:
import time
import serial
from serial.tools import list_ports

PICO_HWID = "2E8A"


def get_pico_port():
    pico_ports = list(list_ports.grep(PICO_HWID))
    if len(pico_ports) == 0:
        raise Exception(
            "No Raspberry Pi Pico was detected. Check to make sure it is plugged in, and that no other programs are accessing it"
        )
    return pico_ports[0].device


print("Here are all the serial devices detected:")
for port in list_ports.comports():
    print(port.device, port.hwid)

port = get_pico_port()
print(f"\nselected port {port} as most likely to have a raspberry pi pico")

Here are all the serial devices detected:
/dev/cu.debug-console n/a
/dev/cu.ShivasBoseFlexSoundLink n/a
/dev/cu.Bluetooth-Incoming-Port n/a


Exception: No Raspberry Pi Pico was detected. Check to make sure it is plugged in, and that no other programs are accessing it

In [ ]:
buffer = bytearray(96 * 96)
img = np.zeros(shape=(96, 96), dtype="uint8")

with serial.Serial(port, timeout=1) as s:
    s.read_until(b"\x55\xAA")
    s.readinto(buffer)
    img.flat[::-1] = buffer

plt.imshow(img, cmap="gray")
plt.show()

In [ ]:
%matplotlib widget

fig, ax = plt.subplots()
render = ax.imshow(img, cmap='gray')
plt.show(block=False)

In [ ]:
try:
    with serial.Serial(port, timeout=1) as s:
        while True:
            s.read_until(b"\x55\xAA")
            s.readinto(buffer)
            img.flat[::-1] = buffer
            frames_pico.append(img.copy())

            img1 = fgbg.apply(frames_pico[-1])
            img2 = cv2.Canny(frames_pico[-1], 200, 100)

            img3 = img1 & img2

            ret, thresh = cv2.threshold(img3, 127, 255, 0)
            c, h = cv2.findContours(thresh.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

            cv2.drawContours(img3, c, -1, (255, 255, 255), 3)
            
            render.set_data(img3)
            fig.canvas.draw()
except KeyboardInterrupt:
    pass

In [6]:
cv2.__version__

'4.11.0'

### <span style="color:red"> Exercise 3</span>

Provide a description of how effectively the motion detector/locator works in a paragraph or two. What methods did your group try? What challenges arose, and were you able to resolve any of them?

<h3 style="background-color:lightblue"> Write Answers for Exercise 3 Below </h3>

The motion detector we implemented using OpenCV actually works very well, as it is able to detect hand movements, fingers, etc, very well. First, we applied the background subtraction and created img1. Then, we wanted to further improve the motion detection by adding an edge detection mechanism. We utilized the Canny() method that detects edges in an image. We generated img2 using edge detection. Then, we found the intersection of img1 and img2 and created img3, which basically implements both background subtraction and edge detection in order to effectively detect motion. However, there was an issue with Canny() because it created dots (circles) instead of actual lines, so it wasn't as visually apealing. As a result, we utilized the findCountours() and drawCountours() methods in order to draw contour lines from the dots/circles that we got for the edges from the Canny() method. The biggest challenge was to be able to research/find ways to improve the motion detector by implementing something else, in addition to the background subtraction. We resolved this by looking at ways to detect edges and utilizing Canny(). Another challenge was that we first did background subtraction and then did edge detection on the same image, so our result/output wasn't as good. In order to resolve this, we decided to apply both separately and then find the intersection/combine them in order to get better/more realistic output. Finally, we had another challenge because the findContours method changed, in terms of inputs and outputs, from opencv version 3.4 to 4.11, but we didn't at first understand the difference until we checked the version numbers. We resolved it by looking at an updated version of the documentation on the OpenCV website and realized that there is only 2 outputs (contours and hiearchy) and that we have to use the drawContours() methods in addition.

## <span style="color:green"> Reflection </span>

Who would win in a fight? A billion lions or all the pokemon? Discuss with your group.
All the pokemon because lions can't fly.